## 1) Download Gene Expression Data

In [2]:
library("spatialLIBD")

Loading required package: SpatialExperiment

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, 

In [13]:
httr::GET("https://raw.githubusercontent.com/LieberInstitute/HumanPilot/master/10X/151508/scalefactors_json.json")

Response [https://raw.githubusercontent.com/LieberInstitute/HumanPilot/master/10X/151508/scalefactors_json.json]
  Date: 2024-02-15 05:25
  Status: 200
  Content-Type: text/plain; charset=utf-8
  Size: 163 B


In [3]:
spe <- fetch_data(type = "spe")

snapshotDate(): 2022-10-31

2024-02-15 14:16:40 loading file /home/edgar/.cache/R/BiocFileCache/295683da82893_Human_DLPFC_Visium_processedData_sce_scran_spatialLIBD.Rdata%3Fdl%3D1

Warning message in open.connection(con, "rb"):
“URL 'https://raw.githubusercontent.com/LieberInstitute/HumanPilot/master/10X/151508/scalefactors_json.json': Timeout of 60 seconds was reached”


ERROR: Error in open.connection(con, "rb"): cannot open the connection to 'https://raw.githubusercontent.com/LieberInstitute/HumanPilot/master/10X/151508/scalefactors_json.json'


In [3]:
spe

class: SpatialExperiment 
dim: 33538 47681 
metadata(0):
assays(2): counts logcounts
rownames(33538): ENSG00000243485 ENSG00000237613 ... ENSG00000277475
  ENSG00000268674
rowData names(9): source type ... gene_search is_top_hvg
colnames(47681): AAACAACGAATAGTTC-1 AAACAAGTATCTCCCA-1 ...
  TTGTTTCCATACAACT-1 TTGTTTGTGTAAATTC-1
colData names(69): sample_id Cluster ... array_row array_col
reducedDimNames(6): PCA TSNE_perplexity50 ... TSNE_perplexity80
  UMAP_neighbors15
mainExpName: NULL
altExpNames(0):
spatialCoords names(2) : pxl_col_in_fullres pxl_row_in_fullres
imgData names(4): sample_id image_id data scaleFactor

In [7]:
data_dir <- "dataset"
if (!file.exists(data_dir)) {
    dir.create(data_dir)
}

In [5]:
saveRDS(spe, file="dataset/spe_all.rds")

## 2) Download Image data for visualization

In [8]:
# Create directory for images
data_dir <- 'dataset'
sample_dir <- 'samples'
download_dir <- file.path(data_dir, sample_dir)
if (!file.exists(download_dir)) {
    dir.create(download_dir)
}

In [11]:
# Function for downloading image data
downloadImageData <- function(sample_id, img.url, scale_factors.url, tissue_positions.url, path) {
    image_dir <- file.path(path, sample_id)
    if (!file.exists(image_dir)) {
        dir.create(image_dir)
    }
    
    curl::curl_download(url = img.url, destfile = file.path(image_dir, "tissue_lowres_image.png"))
    curl::curl_download(url = tissue_positions.url, destfile = file.path(image_dir, basename(path = tissue_positions.url)))
    curl::curl_download(url = scale_factors.url, destfile = file.path(image_dir, basename(path = scale_factors.url)))
}

In [5]:
image_data_source <- cbind(
    c("151507",
    "151672",
    "151673"),
    c("https://spatial-dlpfc.s3.us-east-2.amazonaws.com/images/151507_tissue_lowres_image.png",
     "https://spatial-dlpfc.s3.us-east-2.amazonaws.com/images/151672_tissue_lowres_image.png",
     "https://spatial-dlpfc.s3.us-east-2.amazonaws.com/images/151673_tissue_lowres_image.png"),
    c("https://raw.githubusercontent.com/LieberInstitute/HumanPilot/master/10X/151507/scalefactors_json.json",
     "https://raw.githubusercontent.com/LieberInstitute/HumanPilot/master/10X/151672/scalefactors_json.json",
     "https://raw.githubusercontent.com/LieberInstitute/HumanPilot/master/10X/151673/scalefactors_json.json"),
    c("https://raw.githubusercontent.com/LieberInstitute/HumanPilot/master/10X/151507/tissue_positions_list.txt",
     "https://raw.githubusercontent.com/LieberInstitute/HumanPilot/master/10X/151672/tissue_positions_list.txt",
     "https://raw.githubusercontent.com/LieberInstitute/HumanPilot/master/10X/151673/tissue_positions_list.txt")
)

In [9]:
colnames(image_data_source) <- c("sample_id", "img.url", "scalefactors.url", "tissue_positions.url")

In [12]:
apply(image_data_source, 1, function(row) {
    downloadImageData(row[["sample_id"]], row[["img.url"]], row[["scalefactors.url"]], row[["tissue_positions.url"]], download_dir)
})

ERROR: Error in curl::curl_download(url = tissue_positions.url, destfile = file.path(image_dir, : Timeout was reached: [] Connection timeout after 10000 ms
